In [5]:
import pandas as pd
import torch
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, RMSE, QuantileLoss
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('ai4i2020.csv')

df['cycle_id'] = (df['Tool wear [min]'] == 0).cumsum()
df['group_id'] = df['cycle_id'].astype(str)

long_groups = df['group_id'].value_counts()[df['group_id'].value_counts() >= 30].index
df = df[df['group_id'].isin(long_groups)]

df['time_idx'] = df.groupby('group_id').cumcount()

df['RUL'] = 200 - df['Tool wear [min]']

df = df.drop(columns=['Product ID', 'cycle_id'])
df = df.reset_index(drop=True)

In [3]:
max_prediction_length = 5
max_encoder_length = 20

training = TimeSeriesDataSet(
    df,
    time_idx="time_idx",
    target="RUL",
    group_ids=["group_id"],
    min_encoder_length=1,
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Type"],
    static_reals=["TWF", "HDF", "PWF", "OSF", "RNF"],
    time_varying_known_reals=[
        "Air temperature [K]", 
        "Process temperature [K]", 
        "Rotational speed [rpm]"
    ],
    time_varying_unknown_reals=[
        "Torque [Nm]", 
        "Tool wear [min]"
    ],
    target_normalizer=None,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

In [8]:
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

train_dataloader = training.to_dataloader(train=True, batch_size=32, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=32, num_workers=0)

In [12]:
early_stop = EarlyStopping(monitor="val_loss", patience=10, mode="min")
lr_logger = LearningRateMonitor()

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.01,
    hidden_size=32,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=7,
    loss=QuantileLoss(),
    log_interval=10,
)

trainer = pl.Trainer(
    max_epochs=30,
    accelerator="gpu", #if torch.cuda.is_available() else "cpu",
    enable_model_summary=True,
    callbacks=[early_stop, lr_logger],
    gradient_clip_val=0.1,
)

trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

trainer.save_checkpoint("tft_mining_pm.ckpt")

C:\Users\Kushal Vidyarthi\AppData\Roaming\Python\Python312\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
C:\Users\Kushal Vidyarthi\AppData\Roaming\Python\Python312\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃    ┃ Name                               ┃ Type                            ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0  │ loss                               │ QuantileLoss                    │      0 │ train │     0 │
│ 1  │ logging_metrics                    │ ModuleList                      │      0 │ train │     0 │
│ 2  │ input_embeddings                   │ MultiEmbedding                  │      9 │ train │     0 │
│ 3  │ prescalers                         │ ModuleDict                      │    448 │ train │     0 │
│ 4  │ static_variable_selection          │ VariableSelectionNetwork        │ 16.2 K │ train │     0 │
│ 5  │ encoder_variable_selection         │ VariableSelectionNetwork        │ 11.9 K │ train │     0 │
│ 6  │ decoder_variable_selection         │ VariableSelectionNetwork        │  7.8 K │ train │     0 │
│ 7  │ static_context_variable_selection  │ GatedResidualNetwork            │  4.3 K │ train │     0 │
│ 8  │ static_context_initial_hidden_lstm │ GatedResidualNetwork            │  4.3 K │ train │     0 │
│ 9  │ static_context_initial_cell_lstm   │ GatedResidualNetwork            │  4.3 K │ train │     0 │
│ 10 │ static_context_enrichment          │ GatedResidualNetwork            │  4.3 K │ train │     0 │
│ 11 │ lstm_encoder                       │ LSTM                            │  8.4 K │ train │     0 │
│ 12 │ lstm_decoder                       │ LSTM                            │  8.4 K │ train │     0 │
│ 13 │ post_lstm_gate_encoder             │ GatedLinearUnit                 │  2.1 K │ train │     0 │
│ 14 │ post_lstm_add_norm_encoder         │ AddNorm                         │     64 │ train │     0 │
│ 15 │ static_enrichment                  │ GatedResidualNetwork            │  5.3 K │ train │     0 │
│ 16 │ multihead_attn                     │ InterpretableMultiHeadAttention │  2.6 K │ train │     0 │
│ 17 │ post_attn_gate_norm                │ GateAddNorm                     │  2.2 K │ train │     0 │
│ 18 │ pos_wise_ff                        │ GatedResidualNetwork            │  4.3 K │ train │     0 │
│ 19 │ pre_output_gate_norm               │ GateAddNorm                     │  2.2 K │ train │     0 │
│ 20 │ output_layer                       │ Linear                          │    231 │ train │     0 │
└────┴────────────────────────────────────┴─────────────────────────────────┴────────┴───────┴───────┘

Trainable params: 88.7 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 88.7 K                                                                                               
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 432                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

C:\Users\Kushal 
Vidyarthi\AppData\Roaming\Python\Python312\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434
: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.

C:\Users\Kushal Vidyarthi\AppData\Roaming\Python\Python312\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

C:\Users\Kushal Vidyarthi\AppData\Roaming\Python\Python312\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

C:\Users\Kushal Vidyarthi\AppData\Roaming\Python\Python312\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

C:\Users\Kushal Vidyarthi\AppData\Roaming\Python\Python312\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

C:\Users\Kushal Vidyarthi\AppData\Roaming\Python\Python312\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

`weights_only` was not set, defaulting to `False`.


In [21]:
predictions = tft.predict(val_dataloader, return_y=True)

pred_output = predictions.output.cpu()
actuals = predictions.y[0].cpu() 

mae_val = MAE()(pred_output, actuals)
rmse_val = RMSE()(pred_output, actuals)

print(f"Validation MAE: {mae_val:.4f}")
print(f"Validation RMSE: {rmse_val:.4f}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation MAE: 1.7674
Validation RMSE: 2.2600
